# QUANTUM NEURAL NETWORKS

In [ ]:
pip install tensorflow==2.9.1

In [ ]:
pip install pennylane==0.26

In [ ]:
import pennylane as qml
import numpy as np
import tensorflow as tf

seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
tf.keras.backend.set_floatx('float64')

In [ ]:
from sklearn.datasets import load_breast_cancer

x,y = load_breast_cancer(return_X_y = True)

In [ ]:
from sklearn.model_selection import train_test_split

x_tr, x_test, y_tr, y_test = train_test_split(
    x, y, train_size = 0.8)
x_val, x_test, y_val, y_test = train_test_split(
    x_test, y_test, train_size = 0.5)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 4)

xp_tr = pca.fit_transform(x_tr)
xp_test = pca.transform(x_test)
xp_val = pca.transform(x_val)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
xs_tr = scaler.fit_transform(xp_tr)

In [ ]:
xs_test = scaler.transform(xp_test)
xs_val = scaler.transform(xp_val)

# Restrict all the values to be between -1 and 1.
xs_test = np.clip(xs_test, -1, 1)
xs_val = np.clip(xs_val, -1, 1)

In [ ]:
from itertools import combinations

def ZZFeatureMap(nqubits, data):

    # Number of variables that we will load:
    # could be smaller than the number of qubits.
    nload = min(len(data), nqubits)

    for i in range(nload):
        qml.Hadamard(i)
        qml.RZ(2.0 * data[i], wires = i)

    for pair in list(combinations(range(nload), 2)):
        q0 = pair[0]
        q1 = pair[1]

        qml.CZ(wires = [q0, q1])
        qml.RZ(2.0 * (np.pi - data[q0]) *
            (np.pi - data[q1]), wires = q1)
        qml.CZ(wires = [q0, q1])

def TwoLocal(nqubits, theta, reps = 1):

    for r in range(reps):
        for i in range(nqubits):
            qml.RY(theta[r * nqubits + i], wires = i)
        for i in range(nqubits - 1):
            qml.CNOT(wires = [i, i + 1])

    for i in range(nqubits):
        qml.RY(theta[reps * nqubits + i], wires = i)

In [ ]:
state_0 = [[1], [0]]
M = state_0 * np.conj(state_0).T

In [ ]:
nqubits = 4
dev = qml.device("default.qubit", wires=nqubits)

def qnn_circuit(inputs, theta):
    ZZFeatureMap(nqubits, inputs)
    TwoLocal(nqubits = nqubits, theta = theta, reps = 1)
    return qml.expval(qml.Hermitian(M, wires = [0]))

qnn = qml.QNode(qnn_circuit, dev, interface="tf")

In [ ]:
weights = {"theta": 8}
qlayer = qml.qnn.KerasLayer(qnn, weights, output_dim=1)

In [ ]:
model = tf.keras.models.Sequential([qlayer])

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
model.compile(opt, loss=tf.keras.losses.BinaryCrossentropy())

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(
    monitor = "val_loss", patience = 2, verbose = 1,
    restore_best_weights = True)

In [ ]:
history = model.fit(xs_tr, y_tr, epochs = 10, shuffle = True,
    validation_data = (xs_val, y_val),
    batch_size = 20,
    callbacks = [earlystop])

In [ ]:
import matplotlib.pyplot as plt

def plot_losses(history):
    tr_loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = np.array(range(len(tr_loss))) + 1
    plt.plot(epochs, tr_loss, label = "Training loss")
    plt.plot(epochs, val_loss, label = "Validation loss")
    plt.xlabel("Epoch")
    plt.legend()
    plt.show()

plot_losses(history)

In [ ]:
from sklearn.metrics import accuracy_score

tr_acc = accuracy_score(model.predict(xs_tr) >= 0.5, y_tr)
val_acc = accuracy_score(model.predict(xs_val) >= 0.5, y_val)
test_acc = accuracy_score(model.predict(xs_test) >= 0.5, y_test)

print("Train accuracy:", tr_acc)
print("Validation accuracy:", val_acc)
print("Test accuracy:", test_acc)